<a href="https://colab.research.google.com/github/psriraj17/ML_AirplaneParts/blob/main/airplaneparts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'MyDrive'

Mounted at /content/drive
/content/drive/MyDrive


In [44]:
import pandas as pd  
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


import matplotlib.pyplot as plt

In [3]:

Data =  pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Airplane Parts/Material Size with Hours - Cleansed 20221112.xlsx', header=0)#, quoting = csv.QUOTE_NONE
Data= Data[:-1]
Data = Data.dropna(axis=0)


In [4]:
Data

,Item,Class,Sector,Platform Label,Mtl_Type,Mtl_Form,Mtl_Thick (in),Mtl_Width (in),Mtl_Length (in),Mtl_Makes,Mtl_Axes,Primary - Tmp (hrs),Primary - Est (hrs),Primary - Act (hrs),Deburr - Tmp (hrs),Deburr - Est (hrs),Deburr - Act (hrs)
0,2,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.75,25.00,42.00,2.0,5.0,4.000000,4.000000,5.241667,1.00,1.000000,1.911667
1,3,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.00,11.88,18.00,1.0,5.0,4.000000,4.000000,3.005000,0.50,0.500000,1.038333
2,4,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.75,25.00,42.00,2.0,5.0,4.000000,4.000000,5.177500,0.50,0.500000,1.993333
3,5,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.00,11.88,18.00,1.0,5.0,4.000000,4.000000,2.987500,0.50,0.500000,0.976667
4,6,SNKHS,Mil,6.0,Ti_x000D_\n,EX_x000D_\n,2.00,5.25,68.50,1.0,5.0,4.000000,2.666667,2.825556,0.00,0.000000,0.598125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,1106,LATHE,Mil,19.0,CA_x000D_\n,BR_x000D_\n,4.00,2.00,2.00,1.0,5.0,0.425000,0.482000,0.472718,0.17,0.170000,0.128632
683,1107,SNKHS,Mil,19.0,Al_x000D_\n,PL_x000D_\n,1.25,32.00,46.25,1.0,3.0,5.230000,5.230000,10.074167,2.67,2.670000,1.502000
686,1112,MAG1,MIL,17.0,Al_x000D_\n,PL_x000D_\n,2.75,11.75,40.00,1.0,5.0,1.783333,1.783333,6.967222,1.47,1.470000,2.071667
687,1114,MAG1,Mil,17.0,Al_x000D_\n,PL_x000D_\n,3.00,13.00,39.50,1.0,0.0,1.460000,1.460000,4.531667,1.35,1.350000,2.527500


In [5]:
numerical_data = Data[['Platform Label','Mtl_Thick (in)','Mtl_Width (in)','Mtl_Length (in)','Mtl_Makes','Mtl_Axes']]
#numerical_data = numerical_data[:-1]
numerical_data

,Platform Label,Mtl_Thick (in),Mtl_Width (in),Mtl_Length (in),Mtl_Makes,Mtl_Axes
0,22.0,2.75,25.00,42.00,2.0,5.0
1,22.0,2.00,11.88,18.00,1.0,5.0
2,22.0,2.75,25.00,42.00,2.0,5.0
3,22.0,2.00,11.88,18.00,1.0,5.0
4,6.0,2.00,5.25,68.50,1.0,5.0
...,...,...,...,...,...,...
682,19.0,4.00,2.00,2.00,1.0,5.0
683,19.0,1.25,32.00,46.25,1.0,3.0
686,17.0,2.75,11.75,40.00,1.0,5.0
687,17.0,3.00,13.00,39.50,1.0,0.0


In [48]:
from sklearn.utils.fixes import sklearn
def predict_primaryTemp(numerical_data, target):

  primTemp_train, primTemp_test, target_train, target_test = train_test_split(numerical_data, target, test_size=0.2, random_state=42)

  KNN = KNeighborsRegressor(n_neighbors=2, algorithm='ball_tree')
  KNN.fit(primTemp_train, target_train)

  score = KNN.score(primTemp_test, target_test)
  pred = KNN.predict(primTemp_test)
  
  avg_hrs =  np.sum(np.abs(pred - target_test))/len(target_test)
  print(score)
  print(avg_hrs)

In [49]:
predict_primaryTemp(numerical_data, Data['Primary - Tmp (hrs)'])

0.7463238848067469
0.9968786666666667


In [8]:
from sklearn.utils.fixes import sklearn
def predict_primaryAct(numerical_data, target):

  primAct_train, primAct_test, target_train, target_test = train_test_split(numerical_data, target, test_size=0.2, random_state=42)

  KNN = KNeighborsRegressor(n_neighbors=2, algorithm='ball_tree')
  KNN.fit(primAct_train, target_train)

  score = KNN.score(primAct_test, target_test)
  pred = KNN.predict(primAct_test)
  
  avg_hrs =  np.sum(np.abs(pred - target_test))/len(target_test)
  print(score)
  print(avg_hrs)

In [9]:
predict_primaryAct(numerical_data, Data['Primary - Act (hrs)'])

0.77921164191843
1.3031461593046647


In [10]:
from sklearn.utils.fixes import sklearn
def predict_DeburTemp(numerical_data, target):

  DeburTemp_train, DeburTemp_test, target_train, target_test = train_test_split(numerical_data, target, test_size=0.2, random_state=42)

  svr_poly = KNeighborsRegressor(n_neighbors=2, algorithm='ball_tree')
  svr_poly.fit(DeburTemp_train, target_train)

  score = svr_poly.score(DeburTemp_test, target_test)
  print(score)

In [11]:
predict_DeburTemp(numerical_data, Data['Deburr - Tmp (hrs)'])

0.5883572982901886


In [12]:
from sklearn.utils.fixes import sklearn
def predict_DeburAct(numerical_data, target):

  DeburAct_train, DeburAct_test, target_train, target_test = train_test_split(numerical_data, target, test_size=0.2, random_state=42)

  svr_poly = KNeighborsRegressor(n_neighbors=2, algorithm='ball_tree')
  svr_poly.fit(DeburAct_train, target_train)

  score = svr_poly.score(DeburAct_test, target_test)
  print(score)

In [13]:
predict_DeburAct(numerical_data, Data['Deburr - Act (hrs)'])

0.5140543801515485


In [14]:
combinedTemp = Data['Deburr - Tmp (hrs)'] + Data['Primary - Tmp (hrs)']

In [15]:
from sklearn.utils.fixes import sklearn
def predict_combinedTemp(numerical_data, target):

  num_train, num_test, combinedTemp_train, combinedTemp_test = train_test_split(numerical_data, target, test_size=0.2, random_state=42)

  svr_poly = KNeighborsRegressor(n_neighbors=2, algorithm='ball_tree')
  svr_poly.fit(num_train, combinedTemp_train)

  score = svr_poly.score(num_test, combinedTemp_test)
  print(score)

In [16]:
predict_combinedTemp(numerical_data, combinedTemp)

0.7269946515505437


In [17]:
combinedAct = Data['Primary - Act (hrs)'] + Data['Deburr - Act (hrs)']

In [18]:
from sklearn.utils.fixes import sklearn
def predict_combinedAct(numerical_data, target):

  num_train, num_test, combinedAct_train, combinedAct_test = train_test_split(numerical_data, target, test_size=0.2, random_state=42)

  svr_poly = KNeighborsRegressor(n_neighbors=2, algorithm='ball_tree')
  svr_poly.fit(num_train, combinedAct_train)

  score = svr_poly.score(num_test, combinedAct_test)
  print(score)

In [19]:
predict_combinedAct(numerical_data, combinedAct)

0.795512186033331


Predictions based on XGBoost 

In [20]:
Data

,Item,Class,Sector,Platform Label,Mtl_Type,Mtl_Form,Mtl_Thick (in),Mtl_Width (in),Mtl_Length (in),Mtl_Makes,Mtl_Axes,Primary - Tmp (hrs),Primary - Est (hrs),Primary - Act (hrs),Deburr - Tmp (hrs),Deburr - Est (hrs),Deburr - Act (hrs)
0,2,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.75,25.00,42.00,2.0,5.0,4.000000,4.000000,5.241667,1.00,1.000000,1.911667
1,3,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.00,11.88,18.00,1.0,5.0,4.000000,4.000000,3.005000,0.50,0.500000,1.038333
2,4,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.75,25.00,42.00,2.0,5.0,4.000000,4.000000,5.177500,0.50,0.500000,1.993333
3,5,5AXLS,Com,22.0,Al_x000D_\n,PL_x000D_\n,2.00,11.88,18.00,1.0,5.0,4.000000,4.000000,2.987500,0.50,0.500000,0.976667
4,6,SNKHS,Mil,6.0,Ti_x000D_\n,EX_x000D_\n,2.00,5.25,68.50,1.0,5.0,4.000000,2.666667,2.825556,0.00,0.000000,0.598125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,1106,LATHE,Mil,19.0,CA_x000D_\n,BR_x000D_\n,4.00,2.00,2.00,1.0,5.0,0.425000,0.482000,0.472718,0.17,0.170000,0.128632
683,1107,SNKHS,Mil,19.0,Al_x000D_\n,PL_x000D_\n,1.25,32.00,46.25,1.0,3.0,5.230000,5.230000,10.074167,2.67,2.670000,1.502000
686,1112,MAG1,MIL,17.0,Al_x000D_\n,PL_x000D_\n,2.75,11.75,40.00,1.0,5.0,1.783333,1.783333,6.967222,1.47,1.470000,2.071667
687,1114,MAG1,Mil,17.0,Al_x000D_\n,PL_x000D_\n,3.00,13.00,39.50,1.0,0.0,1.460000,1.460000,4.531667,1.35,1.350000,2.527500


In [21]:
Data['Mtl_Type'] = pd.factorize(Data['Mtl_Type'])[0]
Data['Mtl_Form'] = pd.factorize(Data['Mtl_Form'])[0]
Data['Class'] = pd.factorize(Data['Class'])[0]
Data['Sector'] = pd.factorize(Data['Sector'])[0]

In [22]:
Data

,Item,Class,Sector,Platform Label,Mtl_Type,Mtl_Form,Mtl_Thick (in),Mtl_Width (in),Mtl_Length (in),Mtl_Makes,Mtl_Axes,Primary - Tmp (hrs),Primary - Est (hrs),Primary - Act (hrs),Deburr - Tmp (hrs),Deburr - Est (hrs),Deburr - Act (hrs)
0,2,0,0,22.0,0,0,2.75,25.00,42.00,2.0,5.0,4.000000,4.000000,5.241667,1.00,1.000000,1.911667
1,3,0,0,22.0,0,0,2.00,11.88,18.00,1.0,5.0,4.000000,4.000000,3.005000,0.50,0.500000,1.038333
2,4,0,0,22.0,0,0,2.75,25.00,42.00,2.0,5.0,4.000000,4.000000,5.177500,0.50,0.500000,1.993333
3,5,0,0,22.0,0,0,2.00,11.88,18.00,1.0,5.0,4.000000,4.000000,2.987500,0.50,0.500000,0.976667
4,6,1,1,6.0,1,1,2.00,5.25,68.50,1.0,5.0,4.000000,2.666667,2.825556,0.00,0.000000,0.598125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,1106,7,1,19.0,8,7,4.00,2.00,2.00,1.0,5.0,0.425000,0.482000,0.472718,0.17,0.170000,0.128632
683,1107,1,1,19.0,0,0,1.25,32.00,46.25,1.0,3.0,5.230000,5.230000,10.074167,2.67,2.670000,1.502000
686,1112,2,2,17.0,0,0,2.75,11.75,40.00,1.0,5.0,1.783333,1.783333,6.967222,1.47,1.470000,2.071667
687,1114,2,1,17.0,0,0,3.00,13.00,39.50,1.0,0.0,1.460000,1.460000,4.531667,1.35,1.350000,2.527500


In [23]:
NumCat_data = Data[['Class',	'Sector',	'Platform Label',	'Mtl_Type',	'Mtl_Form',	'Mtl_Thick (in)',	'Mtl_Width (in)',	'Mtl_Length (in)',	'Mtl_Makes'	,'Mtl_Axes']]

In [24]:
learning_rate_list = [0.02, 0.05, 0.1]
max_depth_list = [2, 3, 5]
n_estimators_list = [1000, 2000, 3000]

params_dict = {"learning_rate": learning_rate_list,
               "max_depth": max_depth_list,
               "n_estimators": n_estimators_list}

num_combinations = 1
for v in params_dict.values(): num_combinations *= len(v) 

print(num_combinations)
params_dict

27


{'learning_rate': [0.02, 0.05, 0.1],
 'max_depth': [2, 3, 5],
 'n_estimators': [1000, 2000, 3000]}

In [42]:
def my_roc_auc_score(model, X, y): 
  return roc_auc_score(y, model.predict(X))


model_xgboost_fin = xgboost.XGBClassifier(learning_rate=0.05,
                                          max_depth=2,
                                          n_estimators=5000,
                                          subsample=0.5,
                                          colsample_bytree=0.25,
                                          eval_metric='auc',
                                          verbosity=1,
                                          use_label_encoder=False)


In [45]:
def xgPrimTemp(x, y):

  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

  model = XGBRegressor().fit(x,y)

  predictions = model.predict(x_test)
  print(f"Mean squared error: {mean_squared_error(y_test, predictions)}")


In [46]:
xgPrimTemp(NumCat_data,  Data['Primary - Tmp (hrs)'])

[23:29:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean squared error: 1.3071470975979764
